# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len)

# model modules
from modules.model import Model
model = Model(cfg).to(cfg.device)

# print the number of parameters in the model
print(f'\n{sum(p.numel() for p in model.parameters())/1e3}K parameters\n')
print(model)

ModelConfig(dim=128, device='mps', tokenizer='bpe_v1', vocab_len=8192, num_layers=4, second_resid_norm=False, num_heads=4, head_dim=32, max_seq_len=128, mm_bias=False, pmem_size=336, pmem_count=2, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06) 

 TrainConfig(weight_decay=0.05, batch_size=32, max_iters=100, eval_interval=1, eval_samples=1, checkpoint_interval=None, lr_init=1e-06, lr_max=0.1, lr_min=0.001, warmup_iters=1, final_flat_iters=10, anneal_type='cos', num_restarts=3, T_mult=2)

2067.184K parameters

Model(
  (token_embedder): Embedding(8195, 128)
  (layers): ModuleList(
    (0-3): 4 x Layer(
      (pre_context_norm): Norm()
      (context): ContextMem(
        (k_featurizer): KeyFeatureExtractor(
          (W_k): Linear(in_features=128, out_features=128, bias=False)
          (leaky_avg): LeakyAvg()
        )
        (v_featurizer): ValFeatureExtractor(
          (W_v): Linear(in_features=128, out_features=128, bias=False)
        )
  

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-226173b7dd235c68/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-226173b7dd235c68/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                | 0/100 [00:00<?, ?it/s]/Users/tunadorable/anaconda3/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  1%|▏                       | 1/100 [00:01<02:56,  1.78s/it]

step 0000: lr 0.000000, train loss 126.8932, val loss 126.4831, ppl inf, time elapsed: 1.45 seconds


  2%|▍                       | 2/100 [00:02<01:58,  1.20s/it]

step 0001: lr 0.010000, train loss 122.7961, val loss 122.5900, ppl inf, time elapsed: 2.27 seconds


  3%|▋                       | 3/100 [00:03<01:39,  1.02s/it]

...

step 0042: lr 0.000112, train loss 13.0681, val loss 13.2403, ppl 562559, time elapsed: 44.14 seconds


 44%|██████████             | 44/100 [00:45<00:55,  1.01it/s]

step 0043: lr 0.009998, train loss 12.3136, val loss 12.0247, ppl 166821, time elapsed: 45.14 seconds


 44%|██████████             | 44/100 [00:46<00:58,  1.05s/it]


KeyboardInterrupt: 

# inference test before you decide to save it

In [7]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.9,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a timerelaxedJufencetoldCraBrookefinishjuicywedSinationbalancingzippingsuggestedsettingponiesbeautiinsistedyummiestundersufferingtrafficabyscoopidsJuliasplashingspacesrarefinalvaluableippdivefallendancersdiscoveredalrightcloudsrollCawoolkidspolishesscenouredapolotrunkstraofferadultsAbbieinfantshockedwichscoocompasstootellangryapologibrushesDoggy'sfoldeddarBujoggingKatexactlyGuardianhighllCharlotreturneddays: holesopeningexpercircusOnexttimeinvlonelyThencrumbelfchTwinkleaskedtowncubsscarvesiansetestedsecup, bossyaidwebteamshortwelcomlotionscallslawcrackershelperardenungstarsJaneDanthanksudgedtrafficburnt


# Saving your final model
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [8]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)

NameError: name 'log_data' is not defined